In [32]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import numpy as np

In [33]:
iris = load_iris()
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [34]:
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df["target"] = iris.target
df["flower"] = df.target.apply(lambda x: iris.target_names[x])
df.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,flower
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [35]:
X = df.drop(["flower", "target"], axis = 1)
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
model = SVC(kernel = "rbf", C = 30, gamma = "auto")
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
score

0.9666666666666667

In [36]:
kf = KFold(n_splits = 5)
score_model = []
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    score_model.append(get_score(model, X_train, X_test, y_train, y_test))

In [37]:
score_model

[1.0, 1.0, 0.8666666666666667, 0.9333333333333333, 0.8333333333333334]

In [38]:
cross_val_score(SVC(kernel = "linear", C = 10, gamma = "auto"), X, y, cv = 5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [39]:
cross_val_score(SVC(kernel = "rbf", C = 10, gamma = "auto"), X, y, cv = 5)


array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [40]:
cross_val_score(SVC(kernel = "rbf", C = 20, gamma = "auto"), X, y, cv = 5)


array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [42]:
kernels = ["rbf", "linear"]
C = [1, 10, 20]
avg_scores = {}

for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(SVC(kernel = kval, C = cval, gamma = "auto"), X, y)
        avg_scores[kval + "_" + str(cval)] = np.average(cv_scores)

avg_scores


{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [43]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(SVC(gamma = "auto"), {
    "C":[1, 10, 20],
    "kernel": ["rbf", "linear"]
}, cv = 5, return_train_score = False)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.0040731 , 0.00159984, 0.00159984, 0.00159993, 0.0015995 ,
        0.00187984]),
 'std_fit_time': array([0.0027713 , 0.00319967, 0.00319967, 0.00319986, 0.00319901,
        0.00375967]),
 'mean_score_time': array([0.00063133, 0.0015996 , 0.        , 0.00160007, 0.        ,
        0.        ]),
 'std_score_time': array([0.00085308, 0.0031992 , 0.        , 0.00320015, 0.        ,
        0.        ]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [45]:
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004073,0.002771,0.000631,0.000853,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001600,0.003200,0.001600,0.003199,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001600,0.003200,0.000000,0.000000,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001600,0.003200,0.001600,0.003200,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001600,0.003199,0.000000,0.000000,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.001880,0.003760,0.000000,0.000000,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [47]:
df[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [48]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'feature_names_in_',
 'fit',
 'get_params',
 'inverse_transform',
 

In [49]:
clf.best_score_

0.9800000000000001

In [50]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [53]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(SVC(gamma="auto"), {
    "C":[1, 10, 20],
    "kernel":["rbf", "linear"]
}, cv = 3, return_train_score = False, n_iter = 2)
rs.fit(X, y)
pd.DataFrame(rs.cv_results_)[["param_C", "param_kernel", "mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,10,rbf,0.980000
1,10,linear,0.973333
